 ## Using different clustering methodes from the pytometry package.
 In this example we will evaluate some clustering algorithms for AnnData objects using the pytometry package.

In [1]:
from __future__ import division, print_function

import csv
from tkinter import Tk, filedialog, messagebox
import anndata as ann
import FlowCytometryTools as fct
from matplotlib.pylab import *
from natsort import natsorted
from sklearn.datasets import make_blobs
import os.path
from pathlib import Path
import pytometry.clustering.cluster as cls
import operator
import pandas as pd

Choose a maximum number of centers for your clustering. The clustering will be calculated for 2 centers until the maximum number of centers.

In [2]:
max_center = 4

Lets create some testdata with 8 features and 3 centers. A Cluster object will also be initialized.

In [3]:
elements = range(4)

for data in elements:
    alldata, reference_Label = make_blobs(750, n_features=8, centers=3)
    adata = ann.AnnData(X=alldata)
    cluster = cls.Cluster(adata, max_center)

Now you can cluster your data with one of the four clustering methodes. K-means, c-means, Leiden algorithm and a SOM. Let's start with the Leiden algorithm, using a resolution of 1. Please note that the leiden algotihm doesn't need an a priori number of centers.

In [4]:
cluster.leiden_clustering(resolution=1)

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


3.6072583198547363

The clustering results will be stored in the .obs section, as well es in a variable "cluster.community_cluster".

In [5]:
leiden = cluster.adata.obs['leiden']
leiden = cluster.community_cluster

Now the silhouette score and the gap score will be calculated. To calculate the gap score you will need the inertia first.

In [6]:
cluster.silhouette_score(method='leiden')


inertia = cluster.compute_inertia(label=cluster.community_cluster)
gap = cluster.gap_score(method='leiden', inertia=inertia, n_center=len(np.unique(cluster.community_cluster)))

Now we calculate the k-means clustering for the specified range of centers. The k-means package also contains a methode to calculate the optimal number of centers.

In [7]:
cluster.k_means_clustering()
cluster.calc_optimal_k()
k = cluster.optimal_k

Calculate the silhouette and the gap score for the specified range of centers.

In [8]:
cluster.silhouette_score()
gaps = dict()
for n_center in range(2, max_center+1):
    gaps[str(n_center)] = cluster.gap_score('kmeans', cluster.k_medoids_inertia[str(n_center)], n_center)
    
observation = pd.Categorical(cluster.k_means_clust[str(k)].astype('U'), categories=natsorted(np.unique(cluster.community_cluster).astype('U')))
cluster.adata.obs['kmeans'] = observation

Now we calculate the fuzzy c-means clustering for the specified range of centers. The integer representation is stored in the .obs section, the float (fuzzy) representation is stored in the .uns section. Calculate also the silhouette and the gap score for the specified range of centers.

In [26]:
cluster.fuzzy_clustering()

observation = pd.Categorical(cluster.fuzzy_membership[str(k)], categories=np.unique(cluster.community_cluster))
cluster.adata.obs['fuzzy_int'] = observation
    
cluster.silhouette_score(method='fuzzy')

gaps = dict()
for i in range(2, max_center+1):
    inertia = cluster.compute_inertia(label=cluster.fuzzy_membership[str(i)])
    gap = cluster.gap_score(method='fuzzy', inertia=inertia, n_center=i)
    gaps[str(i)] = gap

cluster.adata.uns['fuzzy_float'] = cluster.fuzzy_cluster[str(cluster.optimal_k)]

IndexError: index 2 is out of bounds for axis 0 with size 2

Lastly we calculate the SOM clustering. Again we calculate the silhouette and the gap score for the specified range of centers.

In [28]:
cluster.som_clustering()
cluster.silhouette_score(method='SOM')
for i in range(2, max_center + 1):
    inertia = cluster.compute_inertia(label=cluster.SOM_label[str(i)])
    gap = cluster.gap_score(method='SOM', inertia=inertia, n_center=len(np.unique(cluster.SOM_label[str(i)])))
    gaps[str(i)] = gap

observation = pd.Categorical(cluster.SOM_label[str(k)], categories=np.unique(k))
cluster.adata.obs['som'] = observation

IndexError: index 1 is out of bounds for axis 0 with size 1

For all four clustering methodes there is also the option to return the runtime.

In [29]:
time = cluster.leiden_clustering(resolution=1)
print(time)

0.0874788761138916


In [30]:
time = cluster.k_means_clustering()
print(time)

0.08636593818664551


In [31]:
time = cluster.fuzzy_clustering()
print(time)

0.030194997787475586


For the SOM clustering you can return the cluster index, the calculated weights and the runtime

In [32]:
index, weights, time = cluster.som_clustering()
print(time)

0.07003378868103027
